In [812]:
%pip install pretty_midi
%pip install tensorflow
%pip install music21


Note: you may need to restart the kernel to use updated packages.
  Obtaining dependency information for protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3 from https://files.pythonhosted.org/packages/51/49/d110f0a43beb365758a252203c43eaaad169fe7749da918869a8c991f726/protobuf-4.25.5-cp37-abi3-macosx_10_9_universal2.whl.metadata
  Using cached protobuf-4.25.5-cp37-abi3-macosx_10_9_universal2.whl.metadata (541 bytes)
Using cached protobuf-4.25.5-cp37-abi3-macosx_10_9_universal2.whl (394 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.0
    Uninstalling protobuf-3.20.0:
      Successfully uninstalled protobuf-3.20.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cond-rnn 3.2.1 requires protobuf<=3.20, but you have protobuf 4.25.5 which is incompatible.
Note: you may need to restart the kernel to us

In [813]:
%pip uninstall fluidsynth -y
%pip install --upgrade fluidsynth 

Found existing installation: fluidsynth 0.2
Uninstalling fluidsynth-0.2:
  Successfully uninstalled fluidsynth-0.2
Note: you may need to restart the kernel to use updated packages.
  Using cached fluidsynth-0.2-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [814]:
import pandas as pd
import numpy as np
import pretty_midi
import pathlib
import collections
import datetime
import glob
import music21
import tensorflow as tf
import fluidsynth


In [815]:
import my_functions
import importlib

In [841]:
importlib.reload(my_functions)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/teovargo/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/0n/vghykfw130n6133zv_3yq7_c0000gn/T/ipykernel_89334/3906374534.py", line 1, in <module>
    importlib.reload(my_functions)
  File "/Users/teovargo/anaconda3/lib/python3.11/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 621, in _exec
  File "<frozen importlib._bootstrap_external>", line 940, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/Users/teovargo/MusicGenProject/my_functions.py", line 141, in <module>
    def plot_piano_roll(notes: pd.DataFrame, count: Optional[int] = None):
                                                    ^^^^^^^^
NameError: name 'Optional' is not defined

During handling of the above exception, another

In [817]:
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

# Sampling rate for audio playback
_SAMPLING_RATE = 16000

In [818]:
data_dir=pathlib.Path('Jazzomat_midi_folder')
filenames = glob.glob(str(data_dir/"*.mid"))
print("Number of Files:", len(filenames))
sorted_filenames = sorted(filenames)
#print(sorted_filenames[:5])
sample_file = sorted_filenames[5]
print(sample_file)


Number of Files: 456
Jazzomat_midi_folder/ArtPepper_Stardust-2_FINAL.mid


In [819]:
pm = pretty_midi.PrettyMIDI(sample_file)


In [ ]:
# This loads up all the notes in the dataset
pitchInst = my_functions.extract_notes()

# Calc the gap between note end and next not start
pitchInst['step'] = pitchInst['start'] - pitchInst['start'].shift(1)

# Fix up the first one cause of the wacky shift problem
pitchInst.loc[0,"step"] = 0

# Calculate the inverval between successive notes
pitchInst['interval'] = (pitchInst['pitch'] - pitchInst['pitch'].shift(1))

# Fix up the first one cause of the wacky shift problem
pitchInst.loc[0, 'interval'] = 0

# apply a contour function
pitchInst['contour'] = pitchInst['interval'].apply(my_functions.contour)

pitchInst['step_norm'] = pitchInst['step']/pitchInst['tempo']
pitchInst['duration_norm'] = pitchInst['duration']/pitchInst['tempo']
n_notes = len(pitchInst)
pitchInst.head()


,melid,pitch,start,end,duration,instrument,key,style,tempo,feel,step,interval,contour,step_norm,duration_norm
0,1,65.0,10.343492,10.482268,0.138776,cl,Bb-maj,COOL,218.8,SWING,0.000000,0.0,0.0,0.000000,0.038055
1,1,63.0,10.637642,10.808889,0.171247,cl,Bb-maj,COOL,218.8,SWING,0.294150,-2.0,-2.0,0.080663,0.046960
2,1,58.0,10.843719,10.924989,0.081270,cl,Bb-maj,COOL,218.8,SWING,0.206077,-5.0,-8.0,0.056511,0.022286
3,1,61.0,10.948209,11.183311,0.235102,cl,Bb-maj,COOL,218.8,SWING,0.104490,3.0,3.0,0.028654,0.064470
4,1,63.0,11.232653,11.363265,0.130612,cl,Bb-maj,COOL,218.8,SWING,0.284444,2.0,2.0,0.078001,0.035817


In [ ]:
seq_length = 20
#REMOVE REMAINDERS FROM TRAINING SET HERE
dfs = dict(tuple(pitchInst.groupby('melid')))

# for i, df in dfs.items():
#     n = len(dfs[i])%seq_length
#     dfs[i].drop(df.tail(n).index, inplace = True)
#     train_neat = pd.concat(df[i], df[i+1])
    

In [822]:
train_sub = pitchInst[pitchInst['melid'] > 56]

key_order = ['pitch', 'contour', 'tempo','step', 'duration']
train_notes = np.stack([train_sub[key] for key in key_order], axis=1)


In [823]:
notes_ds = tf.data.Dataset.from_tensor_slices(train_notes)
notes_ds.element_spec

TensorSpec(shape=(5,), dtype=tf.float64, name=None)

In [824]:
# from tensorFlow MusGen tutorial 
def create_sequences(
    dataset: tf.data.Dataset,
    seq_length: int,
    vocab_size: int,
) -> tf.data.Dataset:
  """Returns TF Dataset of sequence and label examples."""
  seq_length = seq_length+1

  # Take 1 extra for the labels
  windows = dataset.window(seq_length, shift=1, stride=1,
                              drop_remainder=True)

  # `flat_map` flattens the" dataset of datasets" into a dataset of tensors
  flatten = lambda x: x.batch(seq_length, drop_remainder=True)
  sequences = windows.flat_map(flatten)

  # Normalize note pitch 
  def scale_pitch(x):
    x = x/[vocab_size,1.0,1.0,1.0,1.0]
    return x

  # Split the labels
  def split_labels(sequences):
    inputs = sequences[:-1]
    labels_dense = sequences[-1]
    labels = {key:labels_dense[i] for i,key in enumerate(key_order)}

    return scale_pitch(inputs), labels
    #return inputs, labels

  return sequences.map(split_labels, num_parallel_calls=tf.data.AUTOTUNE)

In [825]:
vocab_size = 128
seq_ds = create_sequences(notes_ds, seq_length, vocab_size)
seq_ds.element_spec

(TensorSpec(shape=(20, 5), dtype=tf.float64, name=None),
 {'pitch': TensorSpec(shape=(), dtype=tf.float64, name=None),
  'contour': TensorSpec(shape=(), dtype=tf.float64, name=None),
  'tempo': TensorSpec(shape=(), dtype=tf.float64, name=None),
  'step': TensorSpec(shape=(), dtype=tf.float64, name=None),
  'duration': TensorSpec(shape=(), dtype=tf.float64, name=None)})

In [826]:
batch_size = 64
buffer_size = n_notes - seq_length  # the number of items in the dataset
train_ds = (seq_ds
            .shuffle(buffer_size)
            .batch(batch_size, drop_remainder=True)
            .cache()
            .prefetch(tf.data.experimental.AUTOTUNE))

In [827]:
train_ds.element_spec

(TensorSpec(shape=(64, 20, 5), dtype=tf.float64, name=None),
 {'pitch': TensorSpec(shape=(64,), dtype=tf.float64, name=None),
  'contour': TensorSpec(shape=(64,), dtype=tf.float64, name=None),
  'tempo': TensorSpec(shape=(64,), dtype=tf.float64, name=None),
  'step': TensorSpec(shape=(64,), dtype=tf.float64, name=None),
  'duration': TensorSpec(shape=(64,), dtype=tf.float64, name=None)})

In [828]:
def mse_with_positive_pressure(y_true: tf.Tensor, y_pred: tf.Tensor):
  mse = (y_true - y_pred) ** 2
  positive_pressure = 10 * tf.maximum(-y_pred, 0.0)
  return tf.reduce_mean(mse + positive_pressure)

In [829]:
input_shape = (seq_length, 5)
learning_rate = 0.05

#change input layers here? to suit contour
inputs = tf.keras.Input(input_shape)
x = tf.keras.layers.LSTM(128, return_sequences=True)(inputs)
x = tf.keras.layers.LSTM(64, return_sequences=True)(x)
x = tf.keras.layers.LSTM(32, return_sequences=True)(x)
x = tf.keras.layers.LSTM(16, return_sequences=False)(x)


#and here?
outputs = {
  'pitch': tf.keras.layers.Dense(128, name='pitch')(x),
  'step': tf.keras.layers.Dense(1, name='step')(x),
  'duration': tf.keras.layers.Dense(1, name='duration')(x),
}

model = tf.keras.Model(inputs, outputs)

loss = {
      'pitch':tf.keras.losses.SparseCategoricalCrossentropy(
          from_logits=True),
      'step': mse_with_positive_pressure,
      'duration': mse_with_positive_pressure,
}

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

model.compile(loss=loss, optimizer=optimizer)

model.summary()

Model: "functional_25"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_27      │ (None, 20, 5)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_70 (LSTM)      │ (None, 20, 128)   │     68,608 │ input_layer_27[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_71 (LSTM)      │ (None, 20, 64)    │     49,408 │ lstm_70[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_72 (LSTM)      │ (None, 20, 32)    │     12,416 │ lstm_71[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_73 (LSTM)      │ (None, 16)        │      3,136 │ lstm_72[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ duration (Dense)    │ (None, 1)         │         17 │ lstm_73[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pitch (Dense)       │ (None, 128)       │      2,176 │ lstm_73[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ step (Dense)        │ (None, 1)         │         17 │ lstm_73[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 135,778 (530.38 KB)

 Trainable params: 135,778 (530.38 KB)

 Non-trainable params: 0 (0.00 B)

In [830]:
model.compile(
    loss=loss,
    loss_weights={
        'pitch': 0.8,
        'step': 1.0,
        'duration':1.0,
    },
    optimizer=optimizer,
)

In [831]:
model.evaluate(train_ds, return_dict=True)


2751/2751 ━━━━━━━━━━━━━━━━━━━━ 33s 9ms/step - duration_loss: 0.0558 - loss: 39.8691 - pitch_loss: 3.8932 - step_loss: 35.9201


/Users/teovargo/anaconda3/lib/python3.11/contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


{'duration_loss': 0.05423472076654434,
 'loss': 35.97816848754883,
 'pitch_loss': 3.8918087482452393,
 'step_loss': 32.0321044921875}

In [832]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath='./training_checkpoints/ckpt_{epoch}.weights.h5',
        save_weights_only=True),
    tf.keras.callbacks.EarlyStopping(
        monitor='loss',
        patience=5,
        verbose=1,
        restore_best_weights=True),
]

In [833]:
%%time
epochs = 50

history = model.fit(
    train_ds,
    epochs=epochs,
    callbacks=callbacks,
)

Epoch 1/50
2751/2751 ━━━━━━━━━━━━━━━━━━━━ 78s 27ms/step - duration_loss: 0.0433 - loss: 38.1622 - pitch_loss: 2.7368 - step_loss: 35.3820
Epoch 2/50
2751/2751 ━━━━━━━━━━━━━━━━━━━━ 87s 31ms/step - duration_loss: 0.0282 - loss: 38.1079 - pitch_loss: 2.7152 - step_loss: 35.3644
Epoch 3/50
2751/2751 ━━━━━━━━━━━━━━━━━━━━ 84s 31ms/step - duration_loss: 0.0284 - loss: 38.1087 - pitch_loss: 2.7152 - step_loss: 35.3651
Epoch 4/50
2751/2751 ━━━━━━━━━━━━━━━━━━━━ 92s 34ms/step - duration_loss: 0.0284 - loss: 38.1087 - pitch_loss: 2.7151 - step_loss: 35.3651
Epoch 5/50
2751/2751 ━━━━━━━━━━━━━━━━━━━━ 69s 25ms/step - duration_loss: 0.0284 - loss: 38.1087 - pitch_loss: 2.7151 - step_loss: 35.3651
Epoch 6/50
2751/2751 ━━━━━━━━━━━━━━━━━━━━ 67s 24ms/step - duration_loss: 0.0284 - loss: 38.1087 - pitch_loss: 2.7151 - step_loss: 35.3651
Epoch 7/50
2751/2751 ━━━━━━━━━━━━━━━━━━━━ 66s 24ms/step - duration_loss: 0.0284 - loss: 38.1087 - pitch_loss: 2.7151 - step_loss: 35.3651
Epoch 7: early stopping
Restoring 

In [834]:
def predict_next_note(
    notes: np.ndarray,
    model: tf.keras.Model,
    temperature: float = 1.0) -> tuple[int, float, float]:
  """Generates a note as a tuple of (pitch, step, duration), using a trained sequence model."""

  assert temperature > 0

  # Add batch dimension
  inputs = tf.expand_dims(notes, 0)

  predictions = model.predict(inputs)
  pitch_logits = predictions['pitch']
  #contour = predictions['contour']
  step = predictions['step']
  duration = predictions['duration']

  pitch_logits /= temperature 
  pitch = tf.random.categorical(pitch_logits, num_samples=1) 
  pitch = tf.squeeze(pitch, axis=-1) 
  duration = tf.squeeze(duration, axis=-1)
  step = tf.squeeze(step, axis=-1)

  # `step` and `duration` values should be non-negative
  step = tf.maximum(0, step)
  duration = tf.maximum(0, duration)

  return int(pitch), float(step), float(duration)

In [835]:

temperature = 1.5
num_predictions = 60


test_notes = dfs[1]
#test_notes = midi_to_notes(test_file)
# Calculate the inverval between successive notes
#test_notes['interval'] = (test_notes['pitch'] - test_notes['pitch'].shift(1))
# Fix up the first one cause of the wacky shift problem
#test_notes['interval'][0] = 0
# apply a contour function
#test_notes['contour'] = test_notes['interval'].apply(my_functions.contour)
sample_notes = np.stack([test_notes[key] for key in key_order], axis=1)


# The initial sequence of notes; pitch is normalized similar to training sequences
input_notes = (
    sample_notes[:seq_length] / np.array([vocab_size, 1, 1, 1, 1]))

generated_notes = []
prev_start = 0
input_note = input_notes[-1]
tempo = test_notes['tempo'].iloc[-1]
#contour = 0  #initial contour should be last contour of input sequence
for i in range(num_predictions): #THERE ARE PROBLEMS HERE
  pitch, step, duration = predict_next_note(input_notes, model, temperature)
  start = prev_start + step
  end = start + duration
  interval = pitch - input_note[0]
  contour = my_functions.contour(interval)
  input_note = (pitch, contour, tempo ,step, duration)
  generated_notes.append((*input_note, start, end))
  input_notes = np.delete(input_notes, 0, axis=0)
  input_notes = np.append(input_notes , np.expand_dims(input_note, 0), axis=0) 
  prev_start = start
  

generated_notes = pd.DataFrame(
    generated_notes, columns=(*key_order, 'start', 'end'))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 956ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━

In [836]:
test_notes = test_notes[:seq_length]
start_df = test_notes.drop(['interval'], axis=1)
start_df

,melid,pitch,start,end,duration,instrument,key,style,tempo,feel,step,contour,step_norm,duration_norm
0,1,65.0,10.343492,10.482268,0.138776,cl,Bb-maj,COOL,218.8,SWING,0.000000,0.0,0.000000,0.038055
1,1,63.0,10.637642,10.808889,0.171247,cl,Bb-maj,COOL,218.8,SWING,0.294150,-2.0,0.080663,0.046960
2,1,58.0,10.843719,10.924989,0.081270,cl,Bb-maj,COOL,218.8,SWING,0.206077,-8.0,0.056511,0.022286
3,1,61.0,10.948209,11.183311,0.235102,cl,Bb-maj,COOL,218.8,SWING,0.104490,3.0,0.028654,0.064470
4,1,63.0,11.232653,11.363265,0.130612,cl,Bb-maj,COOL,218.8,SWING,0.284444,2.0,0.078001,0.035817
5,1,58.0,11.551927,11.740590,0.188662,cl,Bb-maj,COOL,218.8,SWING,0.319274,-8.0,0.087552,0.051736
6,1,58.0,11.859592,12.341406,0.481814,cl,Bb-maj,COOL,218.8,SWING,0.307664,0.0,0.084369,0.132125
7,1,50.0,14.535692,14.695329,0.159637,cl,Bb-maj,COOL,218.8,SWING,2.676100,-8.0,0.733848,0.043776
8,1,57.0,14.799819,14.944943,0.145125,cl,Bb-maj,COOL,218.8,SWING,0.264127,8.0,0.072430,0.039797
9,1,60.0,14.973968,15.084263,0.110295,cl,Bb-maj,COOL,218.8,SWING,0.174150,3.0,0.047756,0.030245


In [837]:
generated_notes.head()

,pitch,contour,tempo,step,duration,start,end
0,65,16,218.8,0.131046,0.127458,0.131046,0.258504
1,79,16,218.8,0.131046,0.127458,0.262092,0.389550
2,78,-1,218.8,0.131046,0.127458,0.393138,0.520596
3,64,-16,218.8,0.131046,0.127458,0.524184,0.651642
4,69,8,218.8,0.131046,0.127458,0.655230,0.782688


In [838]:
frames = [start_df, generated_notes]
full_sequence = pd.concat(frames)
full_sequence[10:30]


,melid,pitch,start,end,duration,instrument,key,style,tempo,feel,step,contour,step_norm,duration_norm
10,1.0,58.0,15.116190,15.246803,0.130612,cl,Bb-maj,COOL,218.8,SWING,0.142222,-2.0,0.039001,0.035817
11,1.0,55.0,15.284535,15.381769,0.097234,cl,Bb-maj,COOL,218.8,SWING,0.168345,-3.0,0.046164,0.026664
12,1.0,58.0,15.551565,15.641542,0.089977,cl,Bb-maj,COOL,218.8,SWING,0.267029,3.0,0.073226,0.024674
13,1.0,61.0,15.663311,15.946304,0.282993,cl,Bb-maj,COOL,218.8,SWING,0.111746,3.0,0.030643,0.077603
14,1.0,60.0,15.959365,16.110295,0.150930,cl,Bb-maj,COOL,218.8,SWING,0.296054,-1.0,0.081185,0.041388
15,1.0,58.0,16.126259,16.213333,0.087075,cl,Bb-maj,COOL,218.8,SWING,0.166893,-2.0,0.045766,0.023878
16,1.0,60.0,16.232653,16.616327,0.383673,cl,Bb-maj,COOL,218.8,SWING,0.106395,2.0,0.029176,0.105212
17,1.0,58.0,16.657415,16.790930,0.133515,cl,Bb-maj,COOL,218.8,SWING,0.424762,-2.0,0.116479,0.036613
18,1.0,56.0,16.814150,16.950567,0.136417,cl,Bb-maj,COOL,218.8,SWING,0.156735,-2.0,0.042980,0.037409
19,1.0,60.0,16.976689,17.102948,0.126259,cl,Bb-maj,COOL,218.8,SWING,0.162540,4.0,0.044572,0.034623


In [839]:
example_file = 'contourExample.midi'
instrument = pm.instruments[0]
instrument_name = pretty_midi.program_to_instrument_name(instrument.program)
example_pm = my_functions.notes_to_midi(full_sequence.drop('contour', axis=1), out_file=example_file, instrument_name=instrument_name)

In [840]:
def display_audio(pm: pretty_midi.PrettyMIDI, seconds=30):
  waveform = pm.fluidsynth(fs=_SAMPLING_RATE)
  # Take a sample of the generated waveform to mitigate kernel resets
  waveform_short = waveform[:seconds*_SAMPLING_RATE]
  return display.Audio(waveform_short, rate=_SAMPLING_RATE)